In [1]:
import pandas as pd
import cx_Oracle
import os

In [ ]:
def get_fntxt(path):
    '''
    获取路径下面的所有file中的visual_id
    path:文件夹路径
    '''
    ct_dc = {}
    for rtpath,dirs,files in os.walk(path):
        for f in files:#f是文件名
            if f.endswith('.txt'):
                with open(os.path.join(rtpath,f),'r') as fl:
                    ct_dc.setdefault(f,[line[:-1] for line in fl])
    return ct_dc

aries = cx_Oracle.connect('','','CD.ARIES')
dirlist = ['dbscan_center_result']#算法文件夹列表
rootpath = r'C:\Users\ruihuanz\Documents\02-12-result-wo-images'
for dr in dirlist:
    path = os.path.join(rootpath,dr)
    ct_dc = get_fntxt(path)
    print ct_dc
    df_all = []
    for f,ct in ct_dc.iteritems():#f文件名，ct是visual_id列表
        if ct:
            ct = ','.join(["'{}'".format(i) for i in ct])
            sql = """ select dt.bi_board_id,ts.chamber_id,dt.SOCKET_ID,di.visual_id,ts.lot,dt.device_end_date_time
            from a_device_item di join a_device_testing dt on di.di_id = dt.di_id
            join a_testing_session ts on ts.lao_start_ww = dt.lao_start_ww and ts.ts_id = dt.ts_id
            where ts.operation = '7460' and visual_id in ({})""".format(ct)
            df = pd.read_sql(sql,aries)
            df = df.groupby(['BI_BOARD_ID','CHAMBER_ID','SOCKET_ID']).filter(lambda x:len(x) >= 2)
            df_all.append(df)

    df_all = pd.concat(df_all,ignore_index=True)
    df_all.to_csv(dr+'.csv',index=False)
    print '{} lot 数量 {}'.format(dr,len(df_all.drop_duplicates(subset='LOT')))#每个算法下的lot数